SVM Training

In [ ]:
import cv2
import os
import numpy as np
from sklearn import svm
import joblib

gestures = ['one', 'two', 'three']
features, labels = [], []

for label, gesture in enumerate(gestures, start=1):
    folder = f'data/{gesture}/'
    for img_name in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, img_name), 0)
        if img is None:
            continue
        img = cv2.resize(img, (64, 64)).flatten()
        features.append(img)
        labels.append(label)

clf = svm.SVC(kernel='linear')
clf.fit(features, labels)
joblib.dump(clf, 'config/model.pkl')
print("Model trained and saved.")


ROS Publisher Node

In [ ]:
#!/usr/bin/env python3
import rospy
from std_msgs.msg import Int32
import cv2
import joblib

model = joblib.load('/home/pi/dofbot_gesture_control/config/model.pkl')
pub = rospy.Publisher('/gesture_id', Int32, queue_size=10)

def main():
    rospy.init_node('gesture_detector')
    cap = cv2.VideoCapture(0)
    while not rospy.is_shutdown():
        ret, frame = cap.read()
        if not ret:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        roi = gray[100:200, 100:200]
        roi_resized = cv2.resize(roi, (64, 64)).flatten()
        gesture_id = model.predict([roi_resized])[0]
        pub.publish(gesture_id)

        cv2.rectangle(frame, (100, 100), (200, 200), (255, 0, 0), 2)
        cv2.putText(frame, f"Gesture: {gesture_id}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow("Gesture Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


ROS Subscriber Node

In [ ]:
#!/usr/bin/env python3
import rospy
from std_msgs.msg import Int32
from Arm_Lib import Arm_Device
import time

arm = Arm_Device()

def do_behavior(gesture_id):
    if gesture_id == 1:
        rospy.loginfo("Gesture 1: Pick")
        arm.Arm_serial_servo_write(1, 90, 500)
        arm.Arm_serial_servo_write(2, 45, 500)

    elif gesture_id == 2:
        rospy.loginfo("Gesture 2: Wave")
        for _ in range(2):
            arm.Arm_serial_servo_write(5, 30, 300)
            time.sleep(0.3)
            arm.Arm_serial_servo_write(5, 90, 300)
            time.sleep(0.3)

    elif gesture_id == 3:
        rospy.loginfo("Gesture 3: Reset")
        arm.Arm_serial_servo_write(1, 90, 500)
        arm.Arm_serial_servo_write(2, 90, 500)
        arm.Arm_serial_servo_write(3, 90, 500)

def callback(msg):
    do_behavior(msg.data)

def main():
    rospy.init_node('robot_behavior_node')
    rospy.Subscriber('/gesture_id', Int32, callback)
    rospy.spin()

if __name__ == '__main__':
    main()


ROS Launch File

In [ ]:
<launch>
  <node name="gesture_detector" pkg="dofbot_gesture_control" type="gesture_detect.py" output="screen"/>
  <node name="robot_behavior_node" pkg="dofbot_gesture_control" type="robot_behavior.py" output="screen"/>
</launch>


In [ ]:
roscore


In [ ]:
roslaunch dofbot_gesture_control gesture_control.launch
